In [1]:
import numpy as np
import os
import sys

### Questions:

1) What axis to use in batchnorm argument? 

2) Fix imports, have them all from tensorflow not keras that will make things easier.

In [ ]:
Hbbb_ = np.load('/storage1/bz18310/hist_data/Hbbb/Hbbb.npy')
back_ = np.load('/storage1/bz18310/hist_data/SNu/SNu.npy')

In [ ]:
print(np.shape(Hbbb_))
print(np.shape(back_))
Hbbb = Hbbb_[...,None]
back = back_[...,None]
print(np.shape(Hbbb))

In [ ]:
Hbbb_labels = np.ones(np.shape(Hbbb)[0], dtype=int)
back_labels = np.zeros(np.shape(back)[0], dtype=int)

In [ ]:
data = np.concatenate((Hbbb, back))
labels = np.concatenate((Hbbb_labels, back_labels))
print(np.shape(data))
print(np.shape(labels))

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(data, labels, test_size=0.25, shuffle=True)

In [ ]:
#save for later use and cal call back then
np.save('/storage1/bz18310/hist_data/train_test_120x72/train_X.npy', train_X)
np.save('/storage1/bz18310/hist_data/train_test_120x72/train_y.npy', train_y)
np.save('/storage1/bz18310/hist_data/train_test_120x72/test_X.npy', test_X)
np.save('/storage1/bz18310/hist_data/train_test_120x72/test_y.npy', test_y)

In [2]:
import tensorflow as tf
from tensorflow.python.ops import array_ops, math_ops      #for math operations division_no_nan
from tensorflow.keras.layers import *
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch, Hyperband, BayesianOptimization
from sklearn.metrics import roc_curve, auc
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

In [3]:
model = tf.keras.Sequential()

model.add(Conv2D(4, kernel_size=(3,3), input_shape=(120,72,1)))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Conv2D(8, kernel_size=(3,3)))
model.add(MaxPooling2D(pool_size=(2,2), padding='valid'))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(24))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(BatchNormalization(axis=1))
model.add(Activation('sigmoid'))

model.build(input_shape=(None,120,72,1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 118, 70, 4)        40        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 59, 35, 4)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, 59, 35, 4)         236       
_________________________________________________________________
activation (Activation)      (None, 59, 35, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 57, 33, 8)         296       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 16, 8)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 16, 8)         1

In [4]:
push = lambda x: x > .5       #function to check if number greater than .5

def hls_model_accuracy(hls_model, test_X, test_y):                              #function for testing accurcry of hls model
    return accuracy_score(test_y, [push(i) for i in hls_model.predict(test_X)])

def get_timing(build_result):
    return (build_result['BestLatency'], build_result['WorstLatency'],
            build_result['IntervalMin'], build_result['IntervalMax'])


class CustomSensitivityAtSpecificity(tf.keras.metrics.SensitivityAtSpecificity):     #specificity TN/(TN+FP) 
                                                                                
    def __init__(self, specificity, num_thresholds=200, name=None, dtype=None):
        if specificity < 0 or specificity > 1:
            raise ValueError('`specificity` must be in the range [0, 1].')
        self.specificity = specificity
        self.num_thresholds = num_thresholds
        super().__init__(
            specificity, num_thresholds=num_thresholds, name=name, dtype=dtype)  #from the tf.kera.metrics.SensitivityAtSpecificity
                                                                                 #class, creates out object
    def result(self):
        specificities = math_ops.div_no_nan(
        self.true_negatives, self.true_negatives + self.false_positives)   #from keras class super()
        sensitivities = math_ops.div_no_nan(
        self.true_positives, self.true_positives + self.false_negatives)   
        return self._find_max_under_constraint(
                    specificities, sensitivities, math_ops.greater_equal)  #What?

    def get_threshold(self):
        specificities = math_ops.div_no_nan(
        self.true_negatives, self.true_negatives + self.false_positives)
        
        sensitivities = math_ops.div_no_nan(
        self.true_positives, self.true_positives + self.false_negatives)    #Sensitivity TP/(TP+FN)
        
        specs_above_thresh = array_ops.where(math_ops.greater_equal(specificities, self.value))   #What?
        
        return math_ops.reduce_min(array_ops.gather(self.thresholds, specs_above_thresh)).numpy()

def compiler(model_name):
    opt = tf.keras.optimizers.Adam(0.001)
    sensitivity_metric = tf.keras.metrics.SensitivityAtSpecificity(name='sens_at_spec',
                                                                             specificity=0.99925,     
                                                                             num_thresholds=20000)     
    auc_metric = tf.keras.metrics.AUC(name='auc', num_thresholds=200)   
    metrics = ['accuracy', sensitivity_metric, auc_metric]

    model_name.compile(optimizer=opt, loss='binary_crossentropy', metrics=metrics)
    
    return model_name
       
def trainer(model_name, train_X, train_y):     

    model_name.fit(train_X, 
               train_y, 
               epochs=50, 
               verbose=1,
               batch_size=512, 
               validation_split=.2,   
               shuffle=True,
               callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                             patience=5,
                                                             restore_best_weights=True),     
                            pruning_callbacks.UpdatePruningStep()])

In [5]:
model = compiler(model)

In [ ]:
trainer(model, train_X, train_y)

In [6]:
model.load_weights('./120x72 hist models/try_1_copy_of_old_weights.h5')

In [ ]:
#model.save_weights('./120x72 hist models/try_1_copy_of_old_weights.h5')
model.save('./120x72 hist models/try_1_copy_of_old.h5')

In [72]:
model.evaluate(test_X, test_y)

2942/2942 [==============================] - 35s 10ms/step - loss: 0.0520 - accuracy: 0.9831 - sens_at_spec: 0.3435 - auc: 0.9969


[0.05199510604143143,
 0.9830549955368042,
 0.3435222804546356,
 0.9968696236610413]

In [7]:
import hls4ml
from hls4ml.model.profiling import numerical, activations_keras, boxplot

sys.path.append('/usersc/bz18310/previous_notebook/cms-l1-triggers')

from utils.analysis import eff_rate, optimal_eff_rate
from utils.preprocessing import resize
from utils.plotting import *
from utils.hls4ml_helpers import *

In [8]:
def hls4ml_converter(model_name, directory):
        
    config = hls4ml.utils.config_from_keras_model(model_name, granularity='name')
    print("-----------------------------------")
    print("Configuration")
    display(config)
    print("-----------------------------------")
    hls_model = hls4ml.converters.convert_from_keras_model(model_name,
                                                           hls_config=config,   #set configuration of model
                                                           output_dir=directory,  #6bit model save
                                                           fpga_part='xcku15p-ffva1760-2-e',     #What?
                                                            clock_period=(1/.24),      #set clock period??
                                                              io_type='io_stream')     #
    hls_model.compile()
    return hls_model

In [11]:
hls4ml_model = hls4ml_converter(model, '120x72 hist models/hls_try_1_copy_of_old/hls4ml_prj')

Interpreting Sequential
Topology:
Layer name: conv2d_input, layer type: Input
Layer name: conv2d, layer type: Conv2D
  -> Activation (linear), layer name: conv2d
Layer name: max_pooling2d, layer type: MaxPooling2D
Layer name: batch_normalization, layer type: BatchNormalization
Layer name: activation, layer type: Activation
Layer name: conv2d_1, layer type: Conv2D
  -> Activation (linear), layer name: conv2d_1
Layer name: max_pooling2d_1, layer type: MaxPooling2D
Layer name: batch_normalization_1, layer type: BatchNormalization
Layer name: activation_1, layer type: Activation
Layer name: dense, layer type: Dense
  -> Activation (linear), layer name: dense
Layer name: batch_normalization_2, layer type: BatchNormalization
Layer name: activation_2, layer type: Activation
Layer name: dense_1, layer type: Dense
  -> Activation (linear), layer name: dense_1
Layer name: batch_normalization_3, layer type: BatchNormalization
Layer name: activation_3, layer type: Activation
----------------------

{'Model': {'Precision': 'ap_fixed<16,6>',
  'ReuseFactor': 1,
  'Strategy': 'Latency'},
 'LayerName': {'conv2d_input': {'Precision': {'result': 'ap_fixed<16,6>'}},
  'conv2d': {'Precision': {'weight': 'ap_fixed<16,6>',
    'bias': 'ap_fixed<16,6>',
    'result': 'ap_fixed<16,6>'},
   'ReuseFactor': 1},
  'conv2d_linear': {'Precision': 'ap_fixed<16,6>',
   'ReuseFactor': 1,
   'table_size': 1024,
   'table_t': 'ap_fixed<18,8>'},
  'max_pooling2d': {'Precision': 'ap_fixed<16,6>'},
  'batch_normalization': {'Precision': {'scale': 'ap_fixed<16,6>',
    'bias': 'ap_fixed<16,6>'},
   'ReuseFactor': 1},
  'activation': {'Precision': 'ap_fixed<16,6>',
   'ReuseFactor': 1,
   'table_size': 1024,
   'table_t': 'ap_fixed<18,8>'},
  'conv2d_1': {'Precision': {'weight': 'ap_fixed<16,6>',
    'bias': 'ap_fixed<16,6>',
    'result': 'ap_fixed<16,6>'},
   'ReuseFactor': 1},
  'conv2d_1_linear': {'Precision': 'ap_fixed<16,6>',
   'ReuseFactor': 1,
   'table_size': 1024,
   'table_t': 'ap_fixed<18,8>'},

-----------------------------------
Interpreting Sequential
Topology:
Layer name: conv2d_input, layer type: InputLayer, current shape: [[None, 120, 72, 1]]
Layer name: conv2d, layer type: Conv2D, current shape: [[None, 120, 72, 1]]
Layer name: max_pooling2d, layer type: MaxPooling2D, current shape: [[None, 118, 70, 4]]
Layer name: batch_normalization, layer type: BatchNormalization, current shape: [[None, 59, 35, 4]]
Layer name: activation, layer type: Activation, current shape: [[None, 59, 35, 4]]
Layer name: conv2d_1, layer type: Conv2D, current shape: [[None, 59, 35, 4]]
Layer name: max_pooling2d_1, layer type: MaxPooling2D, current shape: [[None, 57, 33, 8]]
Layer name: batch_normalization_1, layer type: BatchNormalization, current shape: [[None, 28, 16, 8]]
Layer name: activation_1, layer type: Activation, current shape: [[None, 28, 16, 8]]
Layer name: dense, layer type: Dense, current shape: [[None, 28, 16, 8]]
Layer name: batch_normalization_2, layer type: BatchNormalization, cu

In [12]:
hls4ml_model.build(csim=False)

Project myproject_prj does not exist. Rerun "hls4ml build -p 120x72 hist models/hls_try_1_copy_of_old/hls4ml_prj".
